# Congestion Charging - Easy

![rel](https://sqlzoo.net/w/images/f/f6/CongestionCharge.png)

camera(**id**, perim)

keeper(**id**, name, address)

vehicle(**id**, keeper)

image(**_camera_**, **whn**, reg)

permit(**_reg_**, **sDate**, chargeType)

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app18-1")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@1a8bd03e

In [2]:
import spark.implicits._
import spark.sqlContext.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

import spark.sqlContext.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@48683778

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val camera = hiveCxt.table("sqlzoo.camera")
val keeper = hiveCxt.table("sqlzoo.keeper")
val vehicle = hiveCxt.table("sqlzoo.vehicle")
val image = hiveCxt.table("sqlzoo.image")
val permit = hiveCxt.table("sqlzoo.permit")

camera: DataFrame = [id: int, perim: string]
keeper: DataFrame = [id: int, name: string ... 1 more field]
vehicle: DataFrame = [id: string, keeper: int]
image: DataFrame = [camera: int, whn: string ... 1 more field]
permit: DataFrame = [reg: string, sdate: string ... 1 more field]

## Sample query

List the vehicles for which "Strenuous, Sam" is the registered keeper. The link between Keepers and Vehicles is via the foreign key specified in the CREATE TABLE vehicle statement. Note the line:

```
 ,FOREIGN KEY(keeper) REFERENCES keeper(id)
```

This will be the basis of our join condition.

## 1.
Show the name and address of the keeper of vehicle SO 02 PSP.

In [5]:
(keeper.join(vehicle.filter(vehicle("id") === "SO 02 PSP"),
             (keeper("id") === vehicle("keeper")))
 .select("name", "address")
 .showHTML())

name,address
"Strenuous, Sam",Surjection Street


## 2.
Show the number of cameras that take images for incoming vehicles.

In [6]:
(camera.filter(col("perim") === "IN")
 .groupBy()
 .agg(count("id").alias("cnt"))
 .showHTML())

cnt
8


## 3.
List the image details taken by Camera 10 before 26 Feb 2007.

In [7]:
(image.filter(image("whn") < "2007-02-26")
 .join(camera, (image("camera") === camera("id")))
 .orderBy("whn", "camera")
 .showHTML())

camera,whn,reg,id,perim
1,2007-02-25 06:10:13.0,SO 02 ASP,1,IN
17,2007-02-25 06:20:01.0,SO 02 ASP,17,null
18,2007-02-25 06:23:40.0,SO 02 ASP,18,null
9,2007-02-25 06:26:04.0,SO 02 ASP,9,OUT
17,2007-02-25 06:57:31.0,SO 02 CSP,17,null
17,2007-02-25 07:00:40.0,SO 02 CSP,17,null
12,2007-02-25 07:04:31.0,SO 02 CSP,12,OUT
5,2007-02-25 07:10:00.0,SO 02 GSP,5,IN
16,2007-02-25 07:13:00.0,SO 02 GSP,16,OUT
2,2007-02-25 07:20:01.0,SO 02 TSP,2,IN


## 4.
List the number of images taken by each camera. Your answer should show how many images have been taken by camera 1, camera 2 etc. The list must NOT include the images taken by camera 15, 16, 17, 18 and 19.

In [8]:
(image.filter(! image("camera").between(15, 19))
 .groupBy("camera")
 .agg(count("reg").alias("cnt"))
 .showHTML())

camera,cnt
12,4
1,1
3,5
5,1
9,8
8,2
10,4
11,3
2,1


## 5.
A number of vehicles have permits that start on 30th Jan 2007. List the name and address for each keeper in alphabetical order without duplication.

In [9]:
(keeper.withColumnRenamed("id", "keeper")
 .join(vehicle, "keeper")
 .join(permit.filter(to_date(permit("sdate")) === "2007-01-30"),
       (vehicle("id") === permit("reg")))
 .select("name", "address")
 .distinct()
 .orderBy("name")
 .showHTML())

name,address
"Ambiguous, Arthur",Absorption Ave.
"Assiduous, Annie",Attribution Alley
"Contiguous, Carol",Circumscription Close
"Strenuous, Sam",Surjection Street


In [10]:
spark.stop()